In [ ]:
from huggingface_hub import login
import os
import sys
import csv
from tqdm import trange
from transformers import AutoModel,AutoTokenizer
FILE_PATH = './QA_results_GT.csv'
os.environ["OPENAI_API_KEY"] = AAA

In [ ]:
ANA_FILE_PATH = './mthp_output.csv'

naiveanswer_LIST = []
lightraganswer_LIST = []
minianswer_LIST = []
QUESTION_LIST = []
GA_LIST = []
filelength = 0
with open(ANA_FILE_PATH, mode='r', encoding='utf-8') as question_file:
    reader = csv.DictReader(question_file)
    for row in reader:
        QUESTION_LIST.append(row['Question'])
        GA_LIST.append(row['Gold Answer'])
        naiveanswer_LIST.append(row['naive'])
        lightraganswer_LIST.append(row['lightrag'])
        minianswer_LIST.append(row['minirag'])
        filelength = filelength+1

In [ ]:
PROMPT = """
Now, I'll give you a question, a gold answer to this question, and three answers provided by different students.

Determine the answer according to the following rules:
If the answer is correct, get 1 point.
If the answer is irrelevant to the question, it will receive 0 points.
If the answer is incorrect, get -1 point.

Return your answer in JSON mode.

For example:

Question:
When does Li Hua arrive to the city?

Gold Answer:
20260105

Answer1: LiHua arrived on the afternoon of January 5th
Answer2: Sorry, there is no information about LiHua's arrival in the information you provided
Answer3: There is no accurate answer in the information you provided, but according to the first information found, LiHua arrived on April 17th

output:
{{
"Score1": 1,
"Score2": 0,
"Score3": -1,
}}



Real data:

Question:
{question}
Gold Answer:
{ga}

Answer1: {naive}
Answer2: {light}
Answer3: {mini}

output:

"""

In [ ]:
#deepseek
from openai import OpenAI
chatbot = OpenAI(api_key=My_deepseek_key, base_url="https://api.deepseek.com")

chat_list = []
for i in range(filelength):
    p = PROMPT.format(question = QUESTION_LIST[i], ga = GA_LIST[i], naive = naiveanswer_LIST[i], light = lightraganswer_LIST[i], mini = minianswer_LIST[i])
    chat_completion = chatbot.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content":p,
            },
            

        ],
        model="deepseek-chat",
        stream = False
    )
    chat_list.append(chat_completion.choices[0].message.content.strip())

In [ ]:
#openai
from openai import OpenAI
from tqdm import trange
chatbot = OpenAI()
chat_list = []
for i in trange(filelength):
    p = PROMPT.format(question = QUESTION_LIST[i], ga = GA_LIST[i], naive = naiveanswer_LIST[i], light = lightraganswer_LIST[i], mini = minianswer_LIST[i])
    chat_completion = chatbot.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content":p,
            },
        ],
        model="gpt-4o",
    )
    chat_list.append(chat_completion.choices[0].message.content.strip())


In [ ]:
import json
import json_repair
chat_score_list = []    
for chat in chat_list:
    try:
        data = json_repair.loads(chat.strip('```json').strip('```'))
        chat_score_list.append(data)
    except:
        chat_score_list.append(0)
        print('Error in chat:', chat)

all_score1 = [data['Score1'] for data in chat_score_list]
all_score2 = [data['Score2'] for data in chat_score_list]
all_score3 = [data['Score3'] for data in chat_score_list]

all_score1_1 = all_score1.count(1)
all_score1_0 = all_score1.count(0)
all_score1_neg = all_score1.count(-1)

all_score2_1 = all_score2.count(1)
all_score2_0 = all_score2.count(0)
all_score2_neg = all_score2.count(-1)

all_score3_1 = all_score3.count(1)
all_score3_0 = all_score3.count(0)
all_score3_neg = all_score3.count(-1)

all = len(all_score1)
print(all_score1_1, all_score1_0, all_score1_neg)
print(all_score2_1, all_score2_0, all_score2_neg)
print(all_score3_1, all_score3_0, all_score3_neg)

print(f"Score1 1: {all_score1_1 / all * 100:.2f}\%, Score1 0: {all_score1_0 / all * 100:.2f}\%, Score1 -1: {all_score1_neg / all * 100:.2f}\%")    
print(f"Score2 1: {all_score2_1 / all * 100:.2f}\%, Score2 0: {all_score2_0 / all * 100:.2f}\%, Score2 -1: {all_score2_neg / all * 100:.2f}\%")
print(f"Score3 1: {all_score3_1 / all * 100:.2f}\%, Score3 0: {all_score3_0 / all * 100:.2f}\%, Score3 -1: {all_score3_neg / all * 100:.2f}\%")